# Term Harmonize - STEP 2 : Exact Matching
#### Author: Ryan Urbanowicz (ryanurb@upenn.edu) 
#### Institution: University of Pennsylvania - Perleman School of Medicine
#### Project: CMREF Data Harmonization 
#### Date: 7/18/19

#### Project Overview:
See the first notebook in this series ('Step_1_Term_Harmonize_Data_Preparation.ipynb') for an overview of this project, these notebooks, the target application, data availability, code dependencies, and our strategy for generalizing the code in these notebooks. 

#### Notebook Summary:
This notebook is meant to perform exact matching, in order to identify the clear, 'top-quality' matches between our available most specific terms in the data, and the lowest level of our chosen standard terminology (in this case, LLT of the MedDRA v21).

***
## Load Python packages required in this notebook

In [11]:
#Load necessary packages.
import pandas as pd
import numpy as np

# Jupyter Notebook Hack: This code ensures that the results of multiple commands within a given cell are all displayed, rather than just the last. 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#Import Progress bar - useful for estimating time to completion of a given mapping task (given that some chunks of code took multiple multiple hours to overnight to run)
from tqdm import tnrange, tqdm_notebook

***
## Load Working Map File and Lowest Level (most specific) Terminology Standard File


### Create general variable names for the target-application-specific names needed across all project notebooks

In [12]:
# Input filename for 'target dataset' (excel file loaded in this application)
target_study_data = 'Combined_MEDHX_TERMS_20studies.xlsx' 

ont_DL1_data = 'LLT.xlsx' # Input filename for ontology file defining all DL1 terms and their codes. 
ont_DL1_name_col = 'llt_name' # column label for DL1 term name
ont_DL1_code_col ='llt_code' # column label for DL2 term code
ont_DL1_cur_col = 'llt_currency' # column label for term currency
ont_DL2_data = 'PT.xlsx' # Input filename for ontology file defining all DL2 terms and their codes. 
ont_DL3_DL2_data = 'HLT_PT.xlsx' # Input filename for ontology file defining connections between DL2 and DL3 term codes. 
ont_DL3_data = 'HLT.xlsx' # Input filename for ontology file defining all DL3 terms and their codes.
ont_DL4_DL3_data = 'HLGT_HLT.xlsx' # Input filename for ontology file defining connections between DL3 and DL4 term codes. 
ont_DL4_data = 'HLGT.xlsx' # Input filename for ontology file defining all DL4 terms and their codes.
ont_DL5_DL4_data = 'SOC_HLGT.xlsx' # Input filename for ontology file defining connections between DL4 and DL5 term codes. 
ont_DL5_data = 'SOC.xlsx' # Input filename for ontology file defining all DL5 terms and their codes.

DL1_FT1 = 'MHTERM' # focus term 1: This term is available over all studies. 
DL1_FT2 = 'LLT_NAME' # focus term 3: an alternative term available for a subset of studies. This one supposedly conforms to the MedDRA standard so we expect it to yield more exact matches. May offer a better match for the lowest level of the standardized terminology.
DL1_FT3 = 'MHMODIFY' # focus term 2: an alternative term available for a subset of studies. May offer a better match for the lowest level of the standardized terminology.

DL2 = 'PT_NAME' # Secondary level terms (i.e. more general than DL1 terms)
DL3 = 'HLT_NAME' # Tertiary level terms (i.e. more general than DL2 terms)
DL4 = 'HLGT_NAME' # Quarternary level terms (i.e. more general than DL3 terms)
DL5 = 'SOC_NAME' # Quinary Level terms (i.e. more general than DL4 terms)

TL1_qual_code_header = 'LLT_map_code' # column name for lowest term level mapping quality code (added to mapping file)
TL1_name_header = 'T_LLT' # column name for the 'mapped' TL1 - term name (added to mapping file)
TL1_code_header = 'T_LLT_CODE' # column name for the 'mapped' TL1 - term code (added to mapping file)

FZ1_FT1 = 'FZMatch_1_'+DL1_FT1 # column name for best FT1 fuzzy match (temporarily added to mapping file)
FZ2_FT1 = 'FZMatch_2_'+DL1_FT1 # column name for second best FT1 fuzzy match (temporarily added to mapping file)
FZ3_FT1 = 'FZMatch_3_'+DL1_FT1 # column name for third best FT1 fuzzy match (temporarily added to mapping file)
FZ4_FT1 = 'FZMatch_4_'+DL1_FT1 # column name for fourth best FT1 fuzzy match (temporarily added to mapping file)
FZ5_FT1 = 'FZMatch_5_'+DL1_FT1 # column name for fifth best FT1 fuzzy match (temporarily added to mapping file)

FZMC = 'FZMatch_Choice_ID_'+DL1_FT1 #column name for the column where manual annotator will enter the number (1-5) indicating the FT1 fuzzy matched term that offers the best match (if a good one is identified)
FZCT = 'FZMatch_Copied_Term' #column name for the column where manual annotator can alternatively manually copy in the MedDRA LLT term that best matches the term information in this row (can come from FT2 or FT3 if term was not identified in FT1)

FZ1_FT2 = 'FZMatch_1_'+DL1_FT2 # column name for best FT2 fuzzy match (temporarily added to mapping file)
FZ2_FT2 = 'FZMatch_2_'+DL1_FT2 # column name for second best FT2 fuzzy match (temporarily added to mapping file)
FZ3_FT2 = 'FZMatch_3_'+DL1_FT2 # column name for third best FT2 fuzzy match (temporarily added to mapping file)
FZ4_FT2 = 'FZMatch_4_'+DL1_FT2 # column name for forth best FT2 fuzzy match (temporarily added to mapping file)
FZ5_FT2 = 'FZMatch_5_'+DL1_FT2 # column name for fifth best FT2 fuzzy match (temporarily added to mapping file)

FZ1_FT3 = 'FZMatch_1_'+DL1_FT3 # column name for best FT3 fuzzy match (temporarily added to mapping file)
FZ2_FT3 = 'FZMatch_2_'+DL1_FT3 # column name for second best FT3 fuzzy match (temporarily added to mapping file)
FZ3_FT3 = 'FZMatch_3_'+DL1_FT3 # column name for third best FT3 fuzzy match (temporarily added to mapping file)
FZ4_FT3 = 'FZMatch_4_'+DL1_FT3 # column name for forth best FT3 fuzzy match (temporarily added to mapping file)
FZ5_FT3 = 'FZMatch_5_'+DL1_FT3 # column name for fifth best FT3 fuzzy match (temporarily added to mapping file)

### Load mapping file from previous notebook

In [13]:
#Load target (tab-delimited) file into a pandas data frame
target_map_file = 'MH_harmonization_map_1.csv' # Input filename - instead of the original target data we are loading our 'in-progress mapping file' saved in the last notebook.
td = pd.read_csv(target_map_file, na_values=' ') #Data loaded so that blank excell cells are 'NA'
td.shape

(28720, 8)

*APPLICATION NOTE: Note that there are 8 total columns currently. This is useful to note for inserting additional columns needed for the mapping process below.*

### Load Lowest Level Terminology Standard File

In [14]:
tl1 = pd.read_excel(ont_DL1_data, sep='\t',na_values=' ')
tl1.shape

#Filter out any non-current low level terms (LLTs) 
tl1 = tl1.loc[tl1[ont_DL1_cur_col] == 'Y'] #column name is application specific.
tl1.shape
#Readjusts the row index values so there are no gaps in the sequence from the row removal (important for indexing later) 
tl1 = tl1.reset_index(drop=True) 

(78808, 11)

(69531, 11)

***
## Insert New Columns for Mapping
Insert columns into the mapping file needed for the exact matching task. These include a 'quality' code that we will use for characterizing the quality of the primary task of mapping the lowest level terms in the data to the lowest level of our terminiology standard (i.e. in this case the MedDRA standard). 

*To adapt this code to other tasks, users may need to specify different column indexes below. We place these new columns after the original data columns.*

In [15]:
td.insert(loc=8,column=TL1_qual_code_header,value='NA') 
td.insert(loc=9,column=TL1_name_header,value='NA') 
td.insert(loc=10,column=TL1_code_header,value='NA') 

***
## Exact Matching of Level 1 Term
In this stage we step through every unique row of terms in the working maping file taken from the original dataset and search for an exact match between any of the available DL1 terms in the row (FT1, FT2, FT3), and the lowest level of our reference standard terminology (i.e. 'ont_DL1_name_col' in this target application that would be the LLTs of MedDRA v21). Exact matching is complete in a 'case-insensitive' manner thanks to the python method called 'casefold'. 

Before completing this task we lay out a coding scheme to describe the quality/confidence of our level 1 term mapping for this entire proceedure. These codes will be entered into the column with the 'TL1_qual_code_header' label. We have developed a custom coding scheme to suit the needs of our target application:

* 0 = Exact Match - DL1_FT1 and DL1_FT2 both match (i.e. MHTERM = LLT_NAME column)
* 1 = Exact Match - DL1_FT1 match (i.e. MHTERM match)
* 2 = Exact Match - DL1_FT2 match (i.e. LLT_NAME match)
* 3 = Exact Match - DL1_FT3 match (i.e. MHMODIFY match)
* 4 = Fuzzy Matching (High Confidence)
* 5 = Fuzzy Matching (Medium Confidence)
* 6 = Unmapped/NA

The exact matching process will determine for each row if an exact match was found, and if so which of the 4 quality codes is representative of the match.  If no exact match is found, no code will be assigned (for the time being). Exact matching occurs in a set of nested search loops. The code moves onto the next row as soon as an exact match is found using any of the three level 1 data terms available. 

Note that this exact matching loop (below) took multiple hours to complete on a single PC for our target application.

In [16]:
data_count = 0
unmatched_count = 0
# Step through each DL1_FT1 (there should be a term value for ever row in our working map file)
for each in tqdm_notebook(td[DL1_FT1], desc='1st loop'):
    matchFound = False
    term_count = 0
    
    # Check each term of the level 1 term standard. 
    for term in tl1[ont_DL1_name_col]: # the column name reference is specific to the loaded MedDRA LLT file
        
        #Check for DL1_FT1 exact match
        if str(term).casefold() == str(each).casefold(): 
            matchFound = True
            td[TL1_name_header][data_count] = term # Map matching term
            td[TL1_code_header][data_count] = tl1.at[term_count,ont_DL1_code_col] # Map matching term code, the column name reference is specific to the loaded MedDRA LLT file
            
            #Now check whether the subject's DL1_FT1 == DL1_FT2 which is highest level of code confidence. 
            if str(each).casefold() == str(td[DL1_FT2][data_count]).casefold(): # Check for LLT/PRT exact match agreement
                #CODE 0 ASSIGNED
                td[TL1_qual_code_header][data_count] = 0 # Save term mapping quality code
            else:
                #CODE 1 ASSIGNED
                td[TL1_qual_code_header][data_count] = 1 # Save term mapping quality code
            break
            
        term_count +=1
    
    if not matchFound: #Check for presence of DL1_FT2 value, and if present, check for exact match.
        if pd.isna(td[DL1_FT2][data_count]): #Check for missing value
            pass
        else:
            term_count = 0
            
            # Check each term of the level 1 term standard. 
            for term in tl1[ont_DL1_name_col]: # the column name reference is specific to the loaded MedDRA LLT file

                if str(term).casefold() == str(td[DL1_FT2][data_count]).casefold(): #Check for PRT exact match
                    matchFound = True
                    td[TL1_name_header][data_count] = term # Map matching term
                    td[TL1_code_header][data_count] = tl1.at[term_count,ont_DL1_code_col] # Map matching term code, the column name reference is specific to the loaded MedDRA LLT file
                    #CODE 2 ASSIGNED
                    td[TL1_qual_code_header][data_count] = 2 # Save term mapping quality code
                    break
                    
                term_count +=1
                
        if not matchFound: #Check for presence of DL1_FT3 value, and if present, check for exact match.
            if pd.isna(td[DL1_FT3][data_count]): #Check for missing value
                pass
            else:
                term_count = 0

                # Check each term of the level 1 term standard. 
                for term in tl1[ont_DL1_name_col]: # the column name reference is specific to the loaded MedDRA LLT file

                    if str(term).casefold() == str(td[DL1_FT3][data_count]).casefold(): #Check for PRT exact match
                        matchFound = True
                        td[TL1_name_header][data_count] = term # Map matching term
                        td[TL1_code_header][data_count] = tl1.at[term_count,ont_DL1_code_col] # Map matching term code, the column name reference is specific to the loaded MedDRA LLT file
                        #CODE 2 ASSIGNED
                        td[TL1_qual_code_header][data_count] = 3 # Save term mapping quality code
                        break

                    term_count +=1

    if not matchFound:
        unmatched_count +=1
        
    data_count += 1
    
print(str(unmatched_count)+ " rows could not be matched exactly.")

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: 


7115 rows could not be matched exactly.


## Summarize Results of Exact Matching
Report a summary of the exact matching results based on quality scores.

In [17]:
td[TL1_qual_code_header].value_counts()

2     13357
NA     7115
0      3922
1      3190
3      1136
Name: LLT_map_code, dtype: int64

*APPLICATION NOTE: A summary of our exact matching progress:*

* Code 0 Count = 3922 (DL1_FT1 exact matches and DL1_FT1 = DL1_FT2)
* Code 1 Count = 3190 (DL1_FT1 exact matches)
* Code 2 Count = 13357 (DL1_FT2 exact matches)
* Code 3 Count = 1136 (DL1_FT3 exact matches)
* Failed to exact match = 7115

***
## Save working map file
Before moving on to the next step of the harmonization pipeline we will save our mapping file 'in progress'. Since a row index was added in the first notebook, we will set index to 'False' below.  

In [18]:
td.to_csv("MH_harmonization_map_2.csv", header=True, index=False)  

## Notebook conclusions
This notebook completed the exact term matching search and mapping, using (case insensitive) exact matching to resolve as many of the terms to be mapped as possible.  This is the one part of this proceedure that can be completly, reproducibly automated. The next notebook will perform fuzzy matching on all term rows where exact matching was unsuccessful.